In [300]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [301]:
from psdm_analysis.io.utils import get_absolute_path_from_project_root, get_absolute_path_from_working_dir
from psdm_analysis.models.input.container.grid import GridContainer


grid_path = "./simple_raw_grid"
sep = ","
grid = GridContainer.from_csv(
    grid_path,
    sep
)

In [303]:
from psdm_analysis.plots.grid import grid_plot


grid_plot(grid)

In [304]:
node_0 = "df97c0d1-379b-417a-a473-8e7fe37da99d"
node_1 = "6a4547a8-630b-46e4-8144-9cd649e67c07"
node_2 = "b7a5be0d-2662-41b2-99c6-3b8121a75e9e"
node_3 = "1dcddd06-f41a-405b-9686-7f7942852196"
node_4 = "e3c3c6a3-c383-4dbb-9b3f-a14125615386"

In [305]:
from uuid import uuid4
from psdm_analysis.models.input.create.participants import create_pvs

from psdm_analysis.models.input.participant.pv import PhotovoltaicPowerPlants

pv_s_rated = [4, 6, 10]

data_dict = {
    "id": ["PV_NS_02", "PV_NS_03", "PV_NS_04"],
	"node": [node_2, node_3, node_4],
    "s_rated": pv_s_rated,
	"azimuth": [90, 0, -90],
	"elevation_angle": [40 , 45, 30]
}

pvs = create_pvs(data_dict)

In [306]:
from re import I
from psdm_analysis.models.input.create.participants import create_storages


data_dict = {
    "id": ["BS_NS_02", "BS_NS_03", "BS_NS_04"],
    "node": [node_2, node_3, node_4],
    "e_storage": [1.5 * pv_power for pv_power in pv_s_rated],
}

bs = create_storages(data_dict)

In [307]:
from psdm_analysis.models.input.create.participants import create_electric_vehicles
from psdm_analysis.models.input.participant.evs import ElectricVehicles


ev_s_rated = [10, 10, 20]
data_dict = {
	"id": ["EV_NS_02", "EV_NS_03", "EV_NS_04"],
	"node": [node_2, node_3, node_4],
	"s_rated": ev_s_rated,
	"e_storage": [80, 70, 100],
	"e_cons": [20, 16, 19],
}

evs = create_electric_vehicles(data_dict)

In [308]:
from pipes import Template
from psdm_analysis.models.enums import ElectricCurrentType
from psdm_analysis.models.input.create.participants import create_ev_charging_stations
from psdm_analysis.models.input.participant.evcs import EvChargingStations, EvcsLocationType
from string import Template

data_dict = {
    "id": ["EVCS_NS_02", "EVCS_NS_03", "EVCS_NS_04"],
    "node": [node_2, node_3, node_4],
    "location_type": [EvcsLocationType.HOME.value, EvcsLocationType.HOME.value, EvcsLocationType.HOME.value],
	"s_rated": [10, 10, 10],
}

evcs = create_ev_charging_stations(data_dict)

In [309]:
evs.filter_by_nodes(node_2).uuid

Index(['7d81b25b-1f2d-447e-a0e4-5396a849b4ea'], dtype='object')

In [310]:
from psdm_analysis.models.input.create.thermal import create_thermal_busses


data_dict = {
    "id": ["TB_NS_02", "TB_NS_03", "TB_NS_04"],
}

tbs = create_thermal_busses(data_dict)

In [311]:
from psdm_analysis.models.input.create.participants import create_heat_pumps


data_dict = {
    "id": ["HP_NS_02", "HP_NS_03", "HP_NS_04"],
    "node": [node_2, node_3, node_4],
    "thermal_bus": list(tbs.uuid),
    "s_rated": [4, 3, 6],
}

hps = create_heat_pumps(data_dict)

In [312]:
from psdm_analysis.models.input.create.thermal import create_thermal_houses


data_dict = {
    "thermal_bus": list(tbs.uuid),
    "id": ["TH_NS_02", "TH_NS_03", "TH_NS_04"],
    "eth_losses": [0.101, 0.121, 0.131],
    "eth_capa": [8.801, 9.121, 12.131],
}

ths = create_thermal_houses(data_dict)

In [313]:
from psdm_analysis.models.input.create.participants import create_loads


load_data_dict = {
	"id": ["L_NS_02", "L_NS_03", "L_NS_04"],
	"node": [node_2, node_3, node_4],
    "s_rated": [3, 3, 3],
	"e_cons_annual": [3000, 3500, 4000],
	"load_profile": ["h0", "h0", "h0"]
}

loads = create_loads(load_data_dict)

In [314]:
from psdm_analysis.models.input.container.participants import SystemParticipantsContainer
from psdm_analysis.models.input.participant.bm import BiomassPlants
from psdm_analysis.models.input.participant.em import EnergyManagementSystems
from psdm_analysis.models.input.participant.fixed_feed_in import FixedFeedIns
from psdm_analysis.models.input.participant.load import Loads
from psdm_analysis.models.input.participant.wec import WindEnergyConverters


participants = SystemParticipantsContainer(
    ems = EnergyManagementSystems.create_empty(),
    loads = loads,
    pvs = pvs,
    storages = bs,
    evs = evs,
    evcs = evcs,
    hps = hps, 
    wecs = WindEnergyConverters.create_empty(),
    fixed_feed_ins= FixedFeedIns.create_empty(),
    biomass_plants=BiomassPlants.create_empty(),
)

In [315]:
node_participants = participants.build_node_participants_map(grid.raw_grid.nodes)

In [316]:
node_connected_assets = {}
for node, nodal_participants in node_participants.items():
    participant_uuids = []
    for current in nodal_participants.to_list():
        if isinstance(current, ElectricVehicles):
            continue
        participant_uuids.extend(list(current.uuid))
    node_connected_assets[node] = participant_uuids

In [317]:

from psdm_analysis.models.input.create.participants import create_energy_management_systems


data_dict = {
    "id": ["EMS_NS_02", "EMS_NS_03", "EMS_NS_04"],
    "node": [node_2, node_3, node_4],
	"connected_assets": [node_connected_assets[node] for node in [node_2, node_3, node_4]],
	"control_strategy": ["self_optimization", "self_optimization", "self_optimization"]
}

emss = create_energy_management_systems(data_dict)

In [318]:
updated_participants = SystemParticipantsContainer(
    emss,
    participants.loads,
    participants.fixed_feed_ins,
    participants.pvs,
    participants.biomass_plants,
    participants.wecs,
    participants.storages,
    participants.evs,
    participants.evcs,
    participants.hps,
)

In [319]:
node_participants_map = updated_participants.build_node_participants_map(grid.raw_grid.nodes)
updated_grid = GridContainer(
    grid.raw_grid,
    updated_participants,
    grid.primary_data,
    node_participants_map
)

In [320]:
from psdm_analysis.models.input.thermal.grid import ThermalGridContainer


tg = ThermalGridContainer(
    tbs,
    ths
)

In [321]:
updated_grid.to_csv(target_grid_path, include_primary_data=False)
tg.to_csv(target_grid_path)

# EV Mobility Points of Interest

In [322]:
public_pois = pd.read_csv("./public_pois.csv", index_col="uuid")
public_pois

,id,size,lat,lon,categoricallocation
uuid,,,,,
2da545dc-a649-4500-9f1f-0f35baec306d,WORK-poi,100.0,51.484328,7.411648,work
3795354b-8eed-4a43-9aaa-9d0df805c168,SUPERMARKET-poi,100.0,51.484328,7.411648,supermarket
6537e3bf-5da3-4846-aaae-e8e9fec2516a,SERVICES-poi,100.0,51.484328,7.411648,services
c970bce7-a542-4388-bc36-1cd2c9e6b0cd,OTHER_SHOP-poi,100.0,51.484328,7.411648,other_shop
198a66cf-a342-471c-ae1d-54e92f3390f0,MEDICINAL-poi,100.0,51.484328,7.411648,medicinal
64e7dccd-9ceb-45ee-9382-ac04e7006713,BBPG-poi,100.0,51.484328,7.411648,bbpg
5aeb7096-d6e3-4219-aa40-f44f1bebd279,RESTAURANT-poi,100.0,51.484328,7.411648,restaurant
257c8e57-9b0a-4ea0-a95f-7f07e2939c9a,CULTURE-poi,100.0,51.484328,7.411648,culture
699bec33-58cc-4c28-baf4-5b0ded6e8e65,SPORTS-poi,100.0,51.484328,7.411648,sports


In [323]:
import pandas as pd 


def create_poi(
    id: str,
    lat: float,
    lon: float,
    categorical_location: str,
    size: float = 1,
    uuid: str = None,
):
    if uuid is None:
        uuid = str(uuid4())
    return pd.Series(
        {
            "id": id,
            "lat": lat,
            "lon": lon,
            "categoricallocation": categorical_location,
            "size": size,
        }
    ).rename(uuid)

def create_poi_mapping(
    poi: str,
    evcs: str,
    evs: list[str]
):
    evs = "".join([f"{ev} " for ev in evs])   
    print(evs)
    return pd.Series(
        {
            "poi": poi,
            "evcs": evcs,
            "evs": evs
        }
    )

pois = public_pois
poi_mappings = None

for node in list(grid.raw_grid.nodes.uuid):
    ev_uuids = list(evs.filter_by_nodes(node).uuid)
    evcs_uuids = list(evcs.filter_by_nodes(node).uuid)
    assert len(evcs_uuids) <= 1
    if evcs_uuids:
        evcs_uuid = evcs_uuids[0]
        if not ev_uuids:
            raise ValueError("EVCS without EV")
        poi = create_poi(
            id="HOME-poi",
            lat=grid.raw_grid.nodes.latitude[node],
            lon=grid.raw_grid.nodes.longitude[node],
            categorical_location="home",
        )
        pois = pd.concat([pois, poi.to_frame().T])
        poi_mapping = create_poi_mapping(
            poi=poi.name,
            evcs=evcs_uuid,
            evs=ev_uuids
        )
        if poi_mappings is None:
            poi_mappings = poi_mapping
        else:
            poi_mappings = pd.concat([poi_mappings, poi_mapping], axis=1)

pois, poi_mappings = pois, poi_mappings.transpose()
pois, poi_mappings


7d81b25b-1f2d-447e-a0e4-5396a849b4ea 
54acf4fa-49ee-4c35-8541-a518eba61c84 
0a085e92-0914-4eeb-8bdd-9353c17bf3f2 


(                                                             id   size   
 2da545dc-a649-4500-9f1f-0f35baec306d                   WORK-poi  100.0  \
 3795354b-8eed-4a43-9aaa-9d0df805c168            SUPERMARKET-poi  100.0   
 6537e3bf-5da3-4846-aaae-e8e9fec2516a               SERVICES-poi  100.0   
 c970bce7-a542-4388-bc36-1cd2c9e6b0cd             OTHER_SHOP-poi  100.0   
 198a66cf-a342-471c-ae1d-54e92f3390f0              MEDICINAL-poi  100.0   
 64e7dccd-9ceb-45ee-9382-ac04e7006713                   BBPG-poi  100.0   
 5aeb7096-d6e3-4219-aa40-f44f1bebd279             RESTAURANT-poi  100.0   
 257c8e57-9b0a-4ea0-a95f-7f07e2939c9a                CULTURE-poi  100.0   
 699bec33-58cc-4c28-baf4-5b0ded6e8e65                 SPORTS-poi  100.0   
 21bedd21-6a82-416f-a19c-55cd1a8de461              RELIGIOUS-poi  100.0   
 215069ee-e0f3-4e5f-8ee7-2e21c028deb9  HOME-poi-without-charging    1.0   
 9ffb0a8c-e8ee-4aee-ba94-1051c458516f                   HOME-poi      1   
 b9e8d029-9589-4c2f-b5dd-

In [324]:
poi_mappings

,poi,evcs,evs
0,9ffb0a8c-e8ee-4aee-ba94-1051c458516f,08367f08-772e-4bc2-b38a-bf8575c8ed62,7d81b25b-1f2d-447e-a0e4-5396a849b4ea
1,b9e8d029-9589-4c2f-b5dd-bb014fe3aa48,4602fd48-b8be-41f2-8514-6e119309f409,54acf4fa-49ee-4c35-8541-a518eba61c84
0,0587abe6-658d-4b21-962b-29140da56839,432d69a8-e8eb-4db0-93ec-3abe61194472,0a085e92-0914-4eeb-8bdd-9353c17bf3f2


In [325]:
pois

,id,size,lat,lon,categoricallocation
2da545dc-a649-4500-9f1f-0f35baec306d,WORK-poi,100.0,51.484328,7.411648,work
3795354b-8eed-4a43-9aaa-9d0df805c168,SUPERMARKET-poi,100.0,51.484328,7.411648,supermarket
6537e3bf-5da3-4846-aaae-e8e9fec2516a,SERVICES-poi,100.0,51.484328,7.411648,services
c970bce7-a542-4388-bc36-1cd2c9e6b0cd,OTHER_SHOP-poi,100.0,51.484328,7.411648,other_shop
198a66cf-a342-471c-ae1d-54e92f3390f0,MEDICINAL-poi,100.0,51.484328,7.411648,medicinal
64e7dccd-9ceb-45ee-9382-ac04e7006713,BBPG-poi,100.0,51.484328,7.411648,bbpg
5aeb7096-d6e3-4219-aa40-f44f1bebd279,RESTAURANT-poi,100.0,51.484328,7.411648,restaurant
257c8e57-9b0a-4ea0-a95f-7f07e2939c9a,CULTURE-poi,100.0,51.484328,7.411648,culture
699bec33-58cc-4c28-baf4-5b0ded6e8e65,SPORTS-poi,100.0,51.484328,7.411648,sports
21bedd21-6a82-416f-a19c-55cd1a8de461,RELIGIOUS-poi,100.0,51.484328,7.411648,religious


In [326]:
from pathlib import Path


pois_path = Path("./simple_grid/pois")
pois_path.mkdir(exist_ok=True)
pois.to_csv(target_poi_path + "/poi.csv", index_label="uuid")
poi_mappings.to_csv(target_poi_path + "/pois_mapping.csv", index=False)

In [327]:
pois

,id,size,lat,lon,categoricallocation
2da545dc-a649-4500-9f1f-0f35baec306d,WORK-poi,100.0,51.484328,7.411648,work
3795354b-8eed-4a43-9aaa-9d0df805c168,SUPERMARKET-poi,100.0,51.484328,7.411648,supermarket
6537e3bf-5da3-4846-aaae-e8e9fec2516a,SERVICES-poi,100.0,51.484328,7.411648,services
c970bce7-a542-4388-bc36-1cd2c9e6b0cd,OTHER_SHOP-poi,100.0,51.484328,7.411648,other_shop
198a66cf-a342-471c-ae1d-54e92f3390f0,MEDICINAL-poi,100.0,51.484328,7.411648,medicinal
64e7dccd-9ceb-45ee-9382-ac04e7006713,BBPG-poi,100.0,51.484328,7.411648,bbpg
5aeb7096-d6e3-4219-aa40-f44f1bebd279,RESTAURANT-poi,100.0,51.484328,7.411648,restaurant
257c8e57-9b0a-4ea0-a95f-7f07e2939c9a,CULTURE-poi,100.0,51.484328,7.411648,culture
699bec33-58cc-4c28-baf4-5b0ded6e8e65,SPORTS-poi,100.0,51.484328,7.411648,sports
21bedd21-6a82-416f-a19c-55cd1a8de461,RELIGIOUS-poi,100.0,51.484328,7.411648,religious
